In [19]:
import pandas as pd
from clickhouse_driver import Client

CH_HOST = 'localhost'
CH_PORT = 9000
CH_USER = 'default'
CH_PASSWORD = ''
CH_DATABASE = 'default'
CH_TABLE = 'kezekte_school'

csv_file = 'kezekte_school_dummy.csv'
df = pd.read_csv(csv_file, encoding='utf-8-sig')
df.head()

,ID,Дата записи,Время записи,Услуга,Учреждение,Регион,Статус,Возраст ученика,Пол ученика,Канал записи,Дата визита,Явился,Причина отмены,Тип учреждения
0,1,2025-02-22,03:34,Зачисление в 1 класс,Школа-лицей №1,Актюбинская,Записан,9,М,Веб,NaN,Нет,NaN,Гимназия
1,2,2024-06-28,08:06,Запись в продлёнку,Школа-лицей №1,г. Астана,Записан,9,М,Мобильное приложение,NaN,Да,NaN,Школа
2,3,2024-10-03,19:40,Запись в продлёнку,Школа-лицей №10,Карагандинская,Обслужен,6,М,Через школу,2024-10-03,Да,NaN,Школа-лицей
3,4,2024-09-13,05:36,Запрос справки,Школа-лицей №3,г. Алматы,Обслужен,11,М,Веб,2024-09-13,Да,NaN,Школа-лицей
4,5,2025-04-01,12:45,Зачисление в 1 класс,Школа-лицей №6,Карагандинская,Обслужен,6,Ж,Веб,2025-04-01,Да,NaN,Школа-лицей


In [20]:
df = df.rename(columns={
    "ID": "id",
    "Дата записи": "registration_date",
    "Время записи": "registration_time",
    "Услуга": "service",
    "Учреждение": "institution",
    "Регион": "region",
    "Статус": "status",
    "Возраст ученика": "student_age",
    "Пол ученика": "student_gender",
    "Канал записи": "channel",
    "Дата визита": "visit_date",
    "Явился": "attended",
    "Причина отмены": "cancel_reason",
    "Тип учреждения": "institution_type"
})


In [21]:
df['registration_date'] = pd.to_datetime(df['registration_date'], errors='coerce').dt.date
df['visit_date'] = pd.to_datetime(df['visit_date'], errors='coerce')

df['visit_date'] = df['visit_date'].dt.date.where(df['visit_date'].notna(), None)

df['student_age'] = pd.to_numeric(df['student_age'], errors='coerce').fillna(0).astype('UInt8')

In [22]:
str_columns = [
    'registration_time', 'service', 'institution', 'region', 'status',
    'student_gender', 'channel', 'attended', 'cancel_reason', 'institution_type'
]

for col in str_columns:
    df[col] = df[col].fillna('').astype(str)

In [23]:
client = Client(host=CH_HOST, port=CH_PORT, user=CH_USER, password=CH_PASSWORD, database=CH_DATABASE)
client.execute(f"DROP TABLE IF EXISTS {CH_TABLE}")
print(f"🗑 Таблица '{CH_TABLE}' удалена (если существовала).")

🗑 Таблица 'kezekte_school' удалена (если существовала).


In [24]:
def create_ch_table():
    client = Client(host=CH_HOST, port=CH_PORT, user=CH_USER, password=CH_PASSWORD, database=CH_DATABASE)

    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {CH_TABLE} (
        id UInt32,
        registration_date Date,
        registration_time String,
        service String,
        institution String,
        region String,
        status String,
        student_age UInt8,
        student_gender String,
        channel String,
        visit_date Nullable(Date),
        attended String,
        cancel_reason String,
        institution_type String
    ) ENGINE = MergeTree()
    ORDER BY id
    """
    client.execute(create_table_sql)

In [25]:
def upload_to_ch():
    client = Client(host=CH_HOST, port=CH_PORT, user=CH_USER, password=CH_PASSWORD, database=CH_DATABASE)
    data = [tuple(row) for row in df.to_numpy()]
    columns = df.columns.tolist()
    client.execute(f"INSERT INTO {CH_TABLE} ({', '.join(columns)}) VALUES", data)
    print(f"Загружено {len(data)} строк в таблицу '{CH_TABLE}'.")

In [26]:
def connect_to_superset():
    print(f"""
Теперь подключите таблицу в Superset:

1. Перейдите в: Data → Datasets
2. Нажмите: + Dataset
3. Выберите источник ClickHouse
4. База данных: {CH_DATABASE}
5. Таблица: {CH_TABLE}
6. Нажмите Save — и можно строить визуализации!
    """)

In [27]:
create_ch_table()
upload_to_ch()
connect_to_superset()

Загружено 10000 строк в таблицу 'kezekte_school'.

Теперь подключите таблицу в Superset:

1. Перейдите в: Data → Datasets
2. Нажмите: + Dataset
3. Выберите источник ClickHouse
4. База данных: default
5. Таблица: kezekte_school
6. Нажмите Save — и можно строить визуализации!
    
